**Michael Labbe** <br>
**06/15/2023** <br>
**Homework 4**

**1. Import the necessary libraries and set up your workbook to connect to your NetId schema (5 pts).** <br>

In [2]:
# The getpass library will prompt the user for a password without echoing.
#  * You will need to install the getpass module if this is the first time you have used it.
import getpass

# Define PostgreSQL database connection parameters
user = input("Username (NetID):")
host = input("Host:") #"postgres-database.chlsqnrqiao3.us-east-1.rds.amazonaws.com"
dbname = "imdb"
port = "5432"

password = getpass.getpass("Password:") # Get pass will prompt the user for a password without echoing.

Username (NetID):mlabbe
Host:postgres-database.chlsqnrqiao3.us-east-1.rds.amazonaws.com
Password:········


**2. Extract the movie_box_office_results.csv file into a Pandas data frame, then load that data into a table called movie_box_office_results in your NetID's schema.** <br>

In [3]:
import pandas as pd

#Connect to the database
db_URI = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}" 

#Read in the data
df = pd.read_csv(r'C:/Users/malab/OneDrive/Documents/Notre Dame/Databases & Security/Datasets/movie_boxoffice_results.csv') 

In [4]:
#Load the data into table within schema
df.to_sql('movie_box_office_results', con=db_URI, schema=user, if_exists='replace', index=False, method='multi') 

#Show the first 5 values in table
pd.read_sql("""select * from movie_box_office_results limit 5""", db_URI) 

,film,year,rotten_tomatoes_pct_critics,rotten_tomatoes_audience_pct,script_type,opening_weekend,domestic_gross,foreign_gross,worldwide_gross,budget_in_million,oscar_winners
0,7 Days in Entebbe,2018,26.0,39.0,based on a true story,1592645,3189220,5465548,8654768,NaN,None
1,A Simple Favor,2018,85.0,77.0,adaptation,16011689,53548586,41285014,94833600,20.0,None
2,Aquaman,2018,65.0,80.0,adaptation,67873522,259720880,681000000,940720880,160.0,None
3,Bad Samaritan,2018,54.0,73.0,original screenplay,1726868,3424744,0,3424744,NaN,None
4,Beatriz At Dinner,2017,75.0,56.0,original screenplay,141959,7115854,0,7115854,NaN,None


**3. Based on the table movie_box_office_results and data in the IMDb schema, design a table called title_financials integrated with IMDb database.** <br>

** Answer: My table for title_financials will be connected to the IMDB database through the title_basics table, utilizing tconst as the primary key. The following columns in movie_box_office_results will apply in our new title_financials table: opening_weekend, domestic_gross, foreign_gross, worldwide_gross, and budget_in_millions. My constraints for this table would include a constraint on all of the aforementioned columns that does not allow any values less than 0. ** <br>

**4. Create and load title_financials table into "mlabbe" schema. Transform and combine the data from my movie_box_office_results and any necessary IMDb tables. <br>**

In [5]:
import psycopg2  # Python database adapter for PostgreSQL database

def execute_sql(sql, echo=True):
    try:
        pg_conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port
        )
    except psycopg2.Error as e:
        error_message = e.pgerror
        print("Error Connecting:", error_message)

    # Set the connection to autocommit (everything is treated as an individual transaction)
    pg_conn.set_session(autocommit=True)
    
    # The cursor is used to execute our ddl statements.
    pg_cursor = pg_conn.cursor() 
    
    try:
        pg_cursor.execute(sql)
        if echo:
            print(sql)
    except psycopg2.Error as e:
        error_message = e.pgerror
        print("SQL Failed:", error_message)
    
    pg_conn.close()


In [6]:
sql = """SELECT TITLE_BASICS.TCONST,
    TITLE_BASICS.PRIMARYTITLE,
    TITLE_BASICS.STARTYEAR,
    MOVIE_BOX_OFFICE_RESULTS.OPENING_WEEKEND, 
    MOVIE_BOX_OFFICE_RESULTS.DOMESTIC_GROSS, 
    MOVIE_BOX_OFFICE_RESULTS.FOREIGN_GROSS, 
    MOVIE_BOX_OFFICE_RESULTS.WORLDWIDE_GROSS, 
    MOVIE_BOX_OFFICE_RESULTS.BUDGET_IN_MILLION 
    FROM IMDB.TITLE_BASICS 
    LEFT JOIN MLABBE.TITLE_FINANCIALS ON TITLE_FINANCIALS.TCONST = TITLE_BASICS.TCONST 
    LEFT JOIN MLABBE.MOVIE_BOX_OFFICE_RESULTS ON MOVIE_BOX_OFFICE_RESULTS.FILM = TITLE_BASICS.PRIMARYTITLE 
    WHERE TITLETYPE = 'movie'""" 

movie_financials = pd.read_sql(sql, db_URI)

In [7]:
movie_financials.rename(columns = {'primarytitle':'film'}, inplace = True)

movie_financials.to_sql("movie_financials", con=db_URI, index=False, method='multi', if_exists='replace')

# pd.read_sql("""select * from MOVIE_BOX_OFFICE_RESULTS""", db_URI)

631938

In [8]:
# movie_financials

In [9]:
sql = """SELECT distinct movie_financials.tconst,
    MOVIE_financials.film,
    MOVIE_BOX_OFFICE_RESULTS.OPENING_WEEKEND, 
    MOVIE_BOX_OFFICE_RESULTS.DOMESTIC_GROSS, 
    MOVIE_BOX_OFFICE_RESULTS.FOREIGN_GROSS, 
    MOVIE_BOX_OFFICE_RESULTS.WORLDWIDE_GROSS, 
    MOVIE_BOX_OFFICE_RESULTS.BUDGET_IN_MILLION 
    FROM mlabbe.movie_financials 
    INNER JOIN MLABBE.MOVIE_BOX_OFFICE_RESULTS ON MOVIE_BOX_OFFICE_RESULTS.film = movie_financials.film
    WHERE MOVIE_BOX_OFFICE_RESULTS.YEAR = MOVIE_FINANCIALS.STARTYEAR""" 

title_financials = pd.read_sql(sql, db_URI)

In [10]:
#title_financials
title_financials.to_sql("title_financials", con=db_URI, index=False, method='multi', if_exists='replace')

InternalError: (psycopg2.errors.DependentObjectsStillExist) cannot drop table title_financials because other objects depend on it
DETAIL:  view financial_features depends on table title_financials
HINT:  Use DROP ... CASCADE to drop the dependent objects too.

[SQL: 
DROP TABLE title_financials]
(Background on this error at: https://sqlalche.me/e/20/2j85)

In [84]:
# Add the primary key
execute_sql("""ALTER TABLE title_financials
ADD CONSTRAINT title_financials_PK PRIMARY KEY (tconst);""")

SQL Failed: ERROR:  could not create unique index "title_financials_pk"
DETAIL:  Key (tconst)=(tt1979388) is duplicated.



In [85]:
sql = """SELECT *
    FROM mlabbe.title_financials 
    WHERE tconst = 'tt1979388'""" 

pd.read_sql(sql, db_URI)

,tconst,film,opening_weekend,domestic_gross,foreign_gross,worldwide_gross,budget_in_million
0,tt1979388,The Good Dinosaur,39155217,8320188,209120551,217440739,200.0
1,tt1979388,The Good Dinosaur,39155217,123087120,209120551,332207671,200.0


In [86]:
# Quick google search identified 332 million in worldwide-gross, making 217 million data entry false. 
execute_sql("""delete from title_financials where tconst = 'tt1979388' and worldwide_gross = 217440739;""")

delete from title_financials where tconst = 'tt1979388' and worldwide_gross = 217440739;


In [87]:
# Add the primary key (2nd Attempt)
execute_sql("""ALTER TABLE title_financials
ADD CONSTRAINT title_financials_PK PRIMARY KEY (tconst);""")

ALTER TABLE title_financials
ADD CONSTRAINT title_financials_PK PRIMARY KEY (tconst);


In [88]:
# Add nonnegative constraints:
execute_sql("""ALTER TABLE title_financials
ADD CONSTRAINT opening_weekend_nonnegative check (opening_weekend >=0);""")
execute_sql("""ALTER TABLE title_financials
ADD CONSTRAINT domestic_gross_nonnegative check (domestic_gross >=0);""")
execute_sql("""ALTER TABLE title_financials
ADD CONSTRAINT foreign_gross_nonnegative check (foreign_gross >=0);""")
execute_sql("""ALTER TABLE title_financials
ADD CONSTRAINT worldwide_gross_nonnegative check (worldwide_gross >=0);""")
execute_sql("""ALTER TABLE title_financials
ADD CONSTRAINT budget_in_million_nonnegative check (budget_in_million >=0);""")

ALTER TABLE title_financials
ADD CONSTRAINT opening_weekend_nonnegative check (opening_weekend >=0);
ALTER TABLE title_financials
ADD CONSTRAINT domestic_gross_nonnegative check (domestic_gross >=0);
ALTER TABLE title_financials
ADD CONSTRAINT foreign_gross_nonnegative check (foreign_gross >=0);
ALTER TABLE title_financials
ADD CONSTRAINT worldwide_gross_nonnegative check (worldwide_gross >=0);
ALTER TABLE title_financials
ADD CONSTRAINT budget_in_million_nonnegative check (budget_in_million >=0);


**5. Create view named financial_features in mlabbe schema for training predictive models that estimate the gross revenue of films.<br>**

In [270]:
execute_sql("""CREATE OR REPLACE VIEW financial_features AS (SELECT 
    (2023 - title_basics.startyear) as ageofmovie,
    title_basics.runtimeminutes,
    title_ratings.averagerating,
    title_ratings.numvotes,
    title_financials.WORLDWIDE_GROSS
    FROM mlabbe.title_financials 
    LEFT JOIN IMDB.TITLE_BASICS ON TITLE_BASICS.TCONST = TITLE_FINANCIALS.TCONST
    INNER JOIN IMDB.title_ratings on title_ratings.tconst = title_basics.TCONST
    WHERE titletype = 'movie');""")

CREATE OR REPLACE VIEW financial_features AS (SELECT 
    (2023 - title_basics.startyear) as ageofmovie,
    title_basics.runtimeminutes,
    title_ratings.averagerating,
    title_ratings.numvotes,
    title_financials.WORLDWIDE_GROSS
    FROM mlabbe.title_financials 
    LEFT JOIN IMDB.TITLE_BASICS ON TITLE_BASICS.TCONST = TITLE_FINANCIALS.TCONST
    INNER JOIN IMDB.title_ratings on title_ratings.tconst = title_basics.TCONST
    WHERE titletype = 'movie');


**6. Explore the relationships in your financial_features view and make a first attempt at a predictive model to estimate the gross revenue of films.<br>**

My dependent variable will be WORLDWIDE_GROSS. My independent variables are as follows: STARTYEAR, RUNTIMEMINUTES, GENRE, AND PRIMARYNAME (for actor/actress). 

In [271]:
import numpy as np
from sklearn.model_selection import train_test_split

In [272]:
financial_features = pd.read_sql(f"""select * from mlabbe.financial_features;""", db_URI)

In [273]:
financial_features = financial_features.dropna()

In [274]:
X = financial_features[['ageofmovie', 'runtimeminutes', 'averagerating', 'numvotes']]
y = financial_features['worldwide_gross']
X.to_numpy()
y.to_numpy()
X = X.apply(lambda iterator: ((iterator.max() - iterator)/(iterator.max() - iterator.min())).round(2))
y = y.transpose()

In [275]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=1842)

In [276]:
from sklearn  import linear_model 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from math import sqrt

# Train the model
model = linear_model.LinearRegression()
model.fit(X, y)
y_hat = model.predict(X_test)

# Model performance
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)
print('Coefficient of determination (R^2): %.3f' % r2_score(y_test, y_hat))
print('Mean squared error (MSE): %.3f'% mean_squared_error(y_test, y_hat))
print('Root mean squared error (RMSE) : %.3f'% sqrt(mean_squared_error(y_test, y_hat)) )

Coefficients: [ 8.79024741e+07 -1.28572156e+08  2.17176409e+08 -1.81476020e+09]
Intercept: 1813542609.4775708
Coefficient of determination (R^2): 0.391
Mean squared error (MSE): 50837423033545408.000
Root mean squared error (RMSE) : 225471557.039


**Evaluating my model, I know that the percentage of varation that is explained by my model is about 40%. This is a good start. Going forward, I would also use the primaryname for each title as well as the director who filmed each title for my next attempt.**